# Homework 6 notes

In [1]:
# These are the standard imports for CS 111. 
# This list may change as the quarter goes on.

import os
import math
import time
import struct
import json
import pandas as pd
import networkx as nx
import numpy as np
import numpy.linalg as npla
import scipy
import scipy.sparse.linalg as spla
from scipy import sparse
from scipy import linalg
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
%matplotlib tk


In [2]:
def pagerank1(E, return_vector = False, max_iters = 1000, tolerance = 1e-6):
    """compute page rank from dense adjacency matrix

    Inputs:
      E: adjacency matrix with links going from cols to rows.
         E is a matrix of 0s and 1s, where E[i,j] = 1 means 
         that web page (vertex) j has a link to web page i.
      return_vector = False: If True, return the eigenvector as well as the ranking.
      max_iters = 1000: Maximum number of power iterations to do.
      tolerance = 1e-6: Stop when the eigenvector norm changes by less than this.
      
    Outputs:
      ranking: Permutation giving the ranking, most important first
      vector (only if return_vector is True): Dominant eigenvector of PageRank matrix

    This computes page rank by the following steps:
    1. Add links from any dangling vertices to all vertices.
    2. Scale the columns to sum to 1.
    3. Add a constant matrix to represent jumping at random 15% of the time.
    4. Find the dominant eigenvector with the power method.
    5. Sort the eigenvector to get the rankings.

    The homework problem due February 22 asks you to rewrite this code so
    it takes input E as a scipy csr_sparse matrix, and then never creates 
    a full matrix or any large matrix other than E.
    """
    
    if type(E) is not np.ndarray:
        print('Warning, converting input from type', type(E), 'to dense array.')
        E = E.toarray()
                
    nnz = np.count_nonzero(E) # This call for sparse E may be different
    outdegree = np.sum(E, 0)  # This call for sparse E may be different
    nrows, n = E.shape

    assert nrows == n, 'E must be square'
    assert np.max(E) == 1 and np.sum(E) == nnz, 'E must contain only zeros and ones'
    
    #  1. Add links from any dangling vertices to all other vertices.
    #     E + F will be the matrix with the added links.

    F = np.zeros((n,n))
    for j in range(n):
        if outdegree[j] == 0:
            F[:,j] = np.ones(n)
            F[j,j] = 0
    
    #  2. Scale the columns to sum to 1.

    A = (E + F) / np.sum(E + F, 0)
    
    #  3. Add a constant matrix to represent jumping at random 15% of the time.

    S = np.ones((n,n)) / n
    m = 0.15
    M = (1 - m) * A + m * S
    
    #  4. Find the dominant eigenvector.
    #  Start with a vector all of whose entries are equal.

    e = np.ones(n)
    v = e / npla.norm(e)

    for iteration in range(max_iters):
        oldv = v
        
        v = M @ v
        eigval = npla.norm(v)
        v = v / eigval
        
        if npla.norm(v - oldv) < tolerance:
            break
    
    if npla.norm(v - oldv) < tolerance:
        print('Dominant eigenvalue is %f after %d iterations.\n' % (eigval, iteration+1))
    else:
        print('Did not converge to tolerance %e after %d iterations.\n' % (tolerance, max_iters))

    # Check that the eigenvector elements are all the same sign, and make them positive
    assert np.all(v > 0) or np.all(v < 0), 'Error: eigenvector is not all > 0 or < 0'
    vector = np.abs(v)
        
    #  5. Sort the eigenvector and reverse the permutation to get the rankings.
    ranking = np.argsort(vector)[::-1]

    if return_vector:
        return ranking, vector
    else:
        return ranking

In [3]:
def pagerank2(E, return_vector = False, max_iters = 1000, tolerance = 1e-6):
    """compute page rank from sparse adjacency matrix

    Inputs:
      E: adjacency matrix with links going from cols to rows.
         E is a matrix of 0s and 1s, where E[i,j] = 1 means 
         that web page (vertex) j has a link to web page i.
      return_vector = False: If True, return the eigenvector as well as the ranking.
      max_iters = 1000: Maximum number of power iterations to do.
      tolerance = 1e-6: Stop when the eigenvector norm changes by less than this.
      
    Outputs:
      ranking: Permutation giving the ranking, most important first.
      vector (only if return_vector is True): Dominant eigenvector of PageRank matrix.

    This computes page rank by the following steps:
    1. Add links from any dangling vertices to all vertices.
    2. Scale the columns to sum to 1.
    3. Add a constant matrix to represent jumping at random 15% of the time.
    4. Find the dominant eigenvector with the power method.
    5. Sort the eigenvector to get the rankings.
    
    This computes the same page rank as pagerank1, but it never creates
    a new matrix as large as E.
    Instead, it computes the matrix-vector product M @ v in the power method
    in several steps corresponding to the steps in converting E to M.
    """

    # sparse and dense matrices do some things differently, sigh...
    if type(E) is not scipy.sparse.csr.csr_matrix:
        print('Warning, converting input from type', type(E), 'to sparse csr_matrix.')
        E = sparse.csr_matrix(E)

    nnz = E.count_nonzero()
    outdegree = np.array(E.sum(axis = 0))[0]
    nrows, n = E.shape

    assert nrows == n, 'E must be square'
    assert np.max(E) == 1 and np.sum(E) == nnz, 'E must contain only zeros and ones'
    
    #  1. Add links from any dangling vertices to all other vertices.
    #     We don't add any actual links or compute the matrix F with full columns, 
    #     but just compute the vector "d" that picks out the nonzero cols of F.
    #     Then, formally, F = (J - I) @ D,
    #     where J is the all-ones matrix, I is the identity, and D is diag(d),
    #     but we don't ever form I or J or D or F explicitly.
    #
    #  2. Scale the columns of E + F to sum to 1.
    #     Again we don't compute a whole matrix, just a vector "t"
    #     for which the columns of (E + F) @ diag(t) sum to 1
    #     so, formally, A = (E + F) @ T,
    #     where F is as above and T = diag(t).
    #     Each element of t is one over a column sum of E or of F.
    #
    #  We do both steps (1) and (2) in the same loop below.
   
    t = np.zeros(n)
    d = np.zeros(n) 
    for j in range(n):
        if outdegree[j] == 0:
            t[j] = 1 / (n-1)
            d[j] = 1
        else:
            t[j] = 1 / outdegree[j]
            d[j] = 0
    
    
    #  3. Add a constant matrix to represent jumping at random 15% of the time.
    #     Again we don't do this explicitly, just get the ingredients for:
    #         M = (1-m) * A + m * J / n

    m = 0.15
    
    #  4. Find the dominant eigenvector.
    #     Here we use the power method, just as in pagerank1, but we
    #     implement the matrix-vector multiplication M @ v in several steps.
    
    #  Start with v as a vector all of whose entries are 1/n.

    e = np.ones(n)
    v = e / npla.norm(e)

    for iteration in range(max_iters):
        oldv = v
             
        # Now  M @ v = (1-m)*(E + F) @ T @ v  +  m/n * J @ v.
        #
        # If we let w = T @ v = v * t, and note that J @ v = np.sum(v)*e, we get
        #   M @ v = (1-m) * E @ w + (1-m) * F @ w + m/n * np.sum(v)*e, 
        # and since F @ w = J @ D @ w - I @ D @ w = np.sum(w*d) * e - w*d, we finally have
        #   M @ v = (1-m)*(E@w) + (1-m)*np.sum(w*d)*e - (1-m)*w*d + (m/n)*np.sum(v)*e,
        # which requires no matrices except E, multiplying the vector w.
        
        w  = v * t                   # * is elementwise multiply; v, t, w are vectors 
        wd = w * d                   # elementwise multiply again; w, d, and wd are vectors
        v1 = (1-m) * (E @ w)         # the only matrix is the original E, times vector w
        v2 = (1-m) * np.sum(wd) * e  # scalar times vector e
        v3 = (1-m) * wd              # scalar times vector w
        v4 = (m/n) * np.sum(v) * e   # scalar times vector e
        v = v1 + v2 - v3 + v4        # adding and subtracting vectors                 
                    
        eigval = npla.norm(v)
        v = v / eigval
        
        if npla.norm(v - oldv) < tolerance:
            break
    
    if npla.norm(v - oldv) < tolerance:
        print('Dominant eigenvalue is %f after %d iterations.\n' % (eigval, iteration+1))
    else:
        print('Did not converge to tolerance %e after %d iterations.\n' % (tolerance, max_iters))

    # Check that the eigenvector elements are all the same sign, and make them positive
    assert np.all(v > 0) or np.all(v < 0), 'Error: eigenvector is not all > 0 or < 0'
    vector = np.abs(v)
        
    #  5. Sort the eigenvector and reverse the permutation to get the rankings.
    ranking = np.argsort(vector)[::-1]

    if return_vector:
        return ranking, vector
    else:
        return ranking

In [4]:
%cd ../Data

/Users/gilbert/Documents/CS_111_2019_Winter/Data


In [5]:
E = np.load('PageRankEG1.npy')
r, v = pagerank1(E, return_vector = True)
print('r =', r)
print('v =', v)

Dominant eigenvalue is 1.000000 after 19 iterations.

r = [0 2 3 1]
v = [0.69648305 0.26828106 0.54477799 0.38230039]


In [6]:
E = np.load('PageRankEG1.npy')
r, v = pagerank2(E, return_vector = True)
print('r =', r)
print('v =', v)

Warning, converting input from type <class 'numpy.ndarray'> to sparse csr_matrix.
Dominant eigenvalue is 1.000000 after 19 iterations.

r = [0 2 3 1]
v = [0.69648305 0.26828106 0.54477799 0.38230039]


In [7]:
E = np.load('PageRankEG3.npy')
sitename = open('PageRankEG3.nodelabels').read().splitlines()
r, v = pagerank1(E, return_vector = True)
print('r[:10] =', r[:10])
print()
for i in range(10):
    print('rank %d is page %3d: %s' % (i, r[i], sitename[r[i]]))

Dominant eigenvalue is 1.000000 after 56 iterations.

r[:10] = [  0   9  41 129  17  14   8  16  45  12]

rank 0 is page   0: http://www.harvard.edu
rank 1 is page   9: http://www.hbs.edu
rank 2 is page  41: http://search.harvard.edu:8765/custom/query.html
rank 3 is page 129: http://www.med.harvard.edu
rank 4 is page  17: http://www.gse.harvard.edu
rank 5 is page  14: http://www.hms.harvard.edu
rank 6 is page   8: http://www.ksg.harvard.edu
rank 7 is page  16: http://www.hsph.harvard.edu
rank 8 is page  45: http://www.gocrimson.com
rank 9 is page  12: http://www.hsdm.med.harvard.edu


In [8]:
matname = 'webGoogle'

print('Loading matrix from %s.npz\n' % matname)
%time E = sparse.load_npz(matname + '.npz')
print('\nDone loading %d-by-%d matrix.' % E.shape)
n = E.shape[0]
 
print('\nStarting pagerank2...')
%time r = pagerank2(E)
print('\nDone with pagerank2:')
print('r[:10] =', r[:10])
print('\nStarting spla.eigs...')
%time d, V = spla.eigs(E, k = min(6, E.shape[0]-2))
print('\nDone with spla.eigs.\n')


Loading matrix from webGoogle.npz

CPU times: user 356 ms, sys: 92.8 ms, total: 448 ms
Wall time: 454 ms

Done loading 916428-by-916428 matrix.

Starting pagerank2...
Dominant eigenvalue is 1.000000 after 71 iterations.

CPU times: user 9.06 s, sys: 409 ms, total: 9.47 s
Wall time: 5.19 s

Done with pagerank2:
r[:10] = [ 34312  96071 412410  50061 506742 295123 553985 357570 285814 899572]

Starting spla.eigs...
CPU times: user 17 s, sys: 370 ms, total: 17.4 s
Wall time: 9.07 s

Done with spla.eigs.



In [9]:
matname = 'PageRankEG3'

print('Loading matrix from %s.npy\n' % matname)
%time E = np.load(matname + '.npy')
print('\nDone loading %d-by-%d matrix.' % E.shape)
print('\nStarting pagerank1...')
%time r = pagerank1(E)
print('\nDone with pagerank1:')
print('r[:10] =', r[:10]) 
print('\nStarting pagerank2...')
%time r = pagerank2(E)
print('\nDone with pagerank2:')
print('r[:10] =', r[:10])
print('\nStarting linalg.eig...')
%time d, V = linalg.eig(E)
print('\nDone with linalg.eig.\n')

Loading matrix from PageRankEG3.npy

CPU times: user 2.25 ms, sys: 10.3 ms, total: 12.6 ms
Wall time: 11.2 ms

Done loading 500-by-500 matrix.

Starting pagerank1...
Dominant eigenvalue is 1.000000 after 56 iterations.

CPU times: user 24.2 ms, sys: 8.57 ms, total: 32.7 ms
Wall time: 21.2 ms

Done with pagerank1:
r[:10] = [  0   9  41 129  17  14   8  16  45  12]

Starting pagerank2...
Warning, converting input from type <class 'numpy.ndarray'> to sparse csr_matrix.
Dominant eigenvalue is 1.000000 after 56 iterations.

CPU times: user 20.7 ms, sys: 847 µs, total: 21.5 ms
Wall time: 13.2 ms

Done with pagerank2:
r[:10] = [  0   9  41 129  17  14   8  16  45  12]

Starting linalg.eig...
CPU times: user 170 ms, sys: 19.6 ms, total: 189 ms
Wall time: 107 ms

Done with linalg.eig.

